In [23]:
import eikon as ek
import pandas as pd
from datetime import datetime,date, timedelta
import os

import psycopg2 as pg

import warnings
warnings.filterwarnings('ignore')

In [24]:
api_key = os.environ.get('eikon_key')
ek.set_app_key(api_key)

### pull min data

In [25]:
t0 = datetime(2022,1,1)
t0

datetime.datetime(2022, 1, 1, 0, 0)

In [36]:
ric = ['FLT.AX','NCM.AX', 'BHP.AX']

In [37]:
df = ek.get_timeseries(
    ric,
    start_date= t0,
    end_date=t0 + timedelta(30), # limits
    interval='minute',
    calendar='tradingdays')

df

Security             FLT.AX                                     NCM.AX  \
Field                  HIGH    LOW   OPEN   CLOSE COUNT  VOLUME   HIGH   
Date                                                                     
2022-01-04 00:01:00   18.11  18.11  18.11   18.11     3      69  24.46   
2022-01-04 00:02:00   18.13  18.11  18.11   18.13    12     667  24.48   
2022-01-04 00:03:00  18.195  18.13  18.13  18.195    46   30284  24.49   
2022-01-04 00:04:00    18.2  18.15   18.2   18.19    50    7528  24.49   
2022-01-04 00:05:00  18.185  18.16  18.18   18.18    13    1430  24.49   
...                     ...    ...    ...     ...   ...     ...    ...   
2022-01-28 04:58:00   15.99  15.94  15.95   15.95   121   10543  21.45   
2022-01-28 04:59:00   15.97  15.93  15.95   15.96   105    8306  21.45   
2022-01-28 05:00:00   16.05  15.95  15.96   16.01   161   13007  21.44   
2022-01-28 05:01:00   16.01   16.0  16.01    16.0     3     300   <NA>   
2022-01-28 05:11:00   15.98  15.98  15.98   15.98   230  943691   21.5   

Security                                                      BHP.AX  \
Field                   LOW    OPEN  CLOSE COUNT   VOLUME       HIGH   
Date                                                                   
2022-01-04 00:01:00   24.45   24.46  24.46    30     4989  37.583784   
2022-01-04 00:02:00   24.45  24.455  24.48    64     3991  37.583784   
2022-01-04 00:03:00   24.48   24.48  24.49    34     3928  37.548168   
2022-01-04 00:04:00   24.48  24.485  24.49    40     2233  37.592688   
2022-01-04 00:05:00  24.475   24.49  24.48    32     1278  37.628304   
...                     ...     ...    ...   ...      ...        ...   
2022-01-28 04:58:00   21.43   21.44  21.44   158    18633  41.554968   
2022-01-28 04:59:00   21.38  21.445  21.39   217    51140  41.554968   
2022-01-28 05:00:00   21.39   21.39   21.4   198    31927    41.6262   
2022-01-28 05:01:00    <NA>    <NA>   <NA>  <NA>     <NA>       <NA>   
2022-01-28 05:11:00    21.5    21.5   21.5   476  5407773  41.777568   

Security                                                               
Field                      LOW       OPEN      CLOSE COUNT     VOLUME  
Date                                                                   
2022-01-04 00:01:00  37.565976  37.583784  37.583784   100      14868  
2022-01-04 00:02:00   37.53036  37.583784   37.53036   109      18775  
2022-01-04 00:03:00  37.521456  37.548168  37.521456    65       6588  
2022-01-04 00:04:00  37.521456  37.521456  37.592688   122      13519  
2022-01-04 00:05:00  37.601592  37.601592    37.6194    65       7004  
...                        ...        ...        ...   ...        ...  
2022-01-28 04:58:00  41.465928  41.465928  41.550516   747     177747  
2022-01-28 04:59:00  41.501544  41.550516  41.501544   565     172699  
2022-01-28 05:00:00  41.474832  41.501544  41.617296   489     203640  
2022-01-28 05:01:00       <NA>       <NA>       <NA>  <NA>       <NA>  
2022-01-28 05:11:00  41.777568  41.777568  41.777568  1861  158883432  

[5422 rows x 18 columns]

### Create a table for each ticker in the list

In [38]:
conn = pg.connect(
    host='localhost',
    user='postgres',
    database='timeseries',
    password='password')

cur = conn.cursor()
conn.autocommit=True

In [39]:
for i in ric:
    table_name = i.replace('.AX','')
    
    cur.execute('''drop table if exists {}'''.format(table_name))

    sql = '''CREATE TABLE IF NOT EXISTS {}(
    time TIMESTAMP,
    open DOUBLE PRECISION,
    high DOUBLE PRECISION,
    low DOUBLE PRECISION,
    close DOUBLE PRECISION,
    volume NUMERIC
    )'''.format(table_name)

    cur.execute(sql)
    print('Created table: {}'.format(table_name))

Created table: FLT
Created table: NCM
Created table: BHP


### Insert data into corresponding tables

In [40]:
for ticker in ric:
    table_name = ticker.replace('.AX','')
    data = df[ticker]
    
    sql = '''INSERT INTO {}(time, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s)'''.format(table_name)  
    
    for idx,row in data.dropna().iterrows():
        cur.execute(sql, (idx, row.OPEN, row.HIGH, row.LOW, row.CLOSE, row.VOLUME))
        print('inserted', table_name, idx)
        
    print('👍 Gucci!')

inserted FLT 2022-01-04 00:01:00
inserted FLT 2022-01-04 00:02:00
inserted FLT 2022-01-04 00:03:00
inserted FLT 2022-01-04 00:04:00
inserted FLT 2022-01-04 00:05:00
inserted FLT 2022-01-04 00:06:00
inserted FLT 2022-01-04 00:07:00
inserted FLT 2022-01-04 00:08:00
inserted FLT 2022-01-04 00:09:00
inserted FLT 2022-01-04 00:10:00
inserted FLT 2022-01-04 00:11:00
inserted FLT 2022-01-04 00:12:00
inserted FLT 2022-01-04 00:13:00
inserted FLT 2022-01-04 00:14:00
inserted FLT 2022-01-04 00:15:00
inserted FLT 2022-01-04 00:16:00
inserted FLT 2022-01-04 00:17:00
inserted FLT 2022-01-04 00:18:00
inserted FLT 2022-01-04 00:19:00
inserted FLT 2022-01-04 00:20:00
inserted FLT 2022-01-04 00:21:00
inserted FLT 2022-01-04 00:22:00
inserted FLT 2022-01-04 00:23:00
inserted FLT 2022-01-04 00:24:00
inserted FLT 2022-01-04 00:25:00
inserted FLT 2022-01-04 00:26:00
inserted FLT 2022-01-04 00:27:00
inserted FLT 2022-01-04 00:28:00
inserted FLT 2022-01-04 00:29:00
inserted FLT 2022-01-04 00:30:00
inserted F

inserted FLT 2022-01-05 01:14:00
inserted FLT 2022-01-05 01:15:00
inserted FLT 2022-01-05 01:16:00
inserted FLT 2022-01-05 01:17:00
inserted FLT 2022-01-05 01:18:00
inserted FLT 2022-01-05 01:19:00
inserted FLT 2022-01-05 01:20:00
inserted FLT 2022-01-05 01:21:00
inserted FLT 2022-01-05 01:22:00
inserted FLT 2022-01-05 01:23:00
inserted FLT 2022-01-05 01:24:00
inserted FLT 2022-01-05 01:25:00
inserted FLT 2022-01-05 01:26:00
inserted FLT 2022-01-05 01:27:00
inserted FLT 2022-01-05 01:28:00
inserted FLT 2022-01-05 01:29:00
inserted FLT 2022-01-05 01:30:00
inserted FLT 2022-01-05 01:31:00
inserted FLT 2022-01-05 01:32:00
inserted FLT 2022-01-05 01:33:00
inserted FLT 2022-01-05 01:34:00
inserted FLT 2022-01-05 01:35:00
inserted FLT 2022-01-05 01:36:00
inserted FLT 2022-01-05 01:37:00
inserted FLT 2022-01-05 01:38:00
inserted FLT 2022-01-05 01:39:00
inserted FLT 2022-01-05 01:40:00
inserted FLT 2022-01-05 01:41:00
inserted FLT 2022-01-05 01:42:00
inserted FLT 2022-01-05 01:43:00
inserted F

inserted FLT 2022-01-06 02:53:00
inserted FLT 2022-01-06 02:54:00
inserted FLT 2022-01-06 02:55:00
inserted FLT 2022-01-06 02:56:00
inserted FLT 2022-01-06 02:57:00
inserted FLT 2022-01-06 02:58:00
inserted FLT 2022-01-06 02:59:00
inserted FLT 2022-01-06 03:00:00
inserted FLT 2022-01-06 03:01:00
inserted FLT 2022-01-06 03:02:00
inserted FLT 2022-01-06 03:03:00
inserted FLT 2022-01-06 03:04:00
inserted FLT 2022-01-06 03:05:00
inserted FLT 2022-01-06 03:06:00
inserted FLT 2022-01-06 03:07:00
inserted FLT 2022-01-06 03:08:00
inserted FLT 2022-01-06 03:09:00
inserted FLT 2022-01-06 03:10:00
inserted FLT 2022-01-06 03:11:00
inserted FLT 2022-01-06 03:12:00
inserted FLT 2022-01-06 03:13:00
inserted FLT 2022-01-06 03:14:00
inserted FLT 2022-01-06 03:15:00
inserted FLT 2022-01-06 03:16:00
inserted FLT 2022-01-06 03:17:00
inserted FLT 2022-01-06 03:18:00
inserted FLT 2022-01-06 03:19:00
inserted FLT 2022-01-06 03:20:00
inserted FLT 2022-01-06 03:21:00
inserted FLT 2022-01-06 03:22:00
inserted F

inserted FLT 2022-01-10 00:51:00
inserted FLT 2022-01-10 00:52:00
inserted FLT 2022-01-10 00:53:00
inserted FLT 2022-01-10 00:54:00
inserted FLT 2022-01-10 00:56:00
inserted FLT 2022-01-10 00:57:00
inserted FLT 2022-01-10 00:58:00
inserted FLT 2022-01-10 00:59:00
inserted FLT 2022-01-10 01:00:00
inserted FLT 2022-01-10 01:01:00
inserted FLT 2022-01-10 01:02:00
inserted FLT 2022-01-10 01:03:00
inserted FLT 2022-01-10 01:04:00
inserted FLT 2022-01-10 01:05:00
inserted FLT 2022-01-10 01:06:00
inserted FLT 2022-01-10 01:07:00
inserted FLT 2022-01-10 01:08:00
inserted FLT 2022-01-10 01:09:00
inserted FLT 2022-01-10 01:10:00
inserted FLT 2022-01-10 01:11:00
inserted FLT 2022-01-10 01:12:00
inserted FLT 2022-01-10 01:13:00
inserted FLT 2022-01-10 01:14:00
inserted FLT 2022-01-10 01:15:00
inserted FLT 2022-01-10 01:16:00
inserted FLT 2022-01-10 01:17:00
inserted FLT 2022-01-10 01:18:00
inserted FLT 2022-01-10 01:19:00
inserted FLT 2022-01-10 01:20:00
inserted FLT 2022-01-10 01:21:00
inserted F

inserted FLT 2022-01-11 02:55:00
inserted FLT 2022-01-11 02:56:00
inserted FLT 2022-01-11 02:57:00
inserted FLT 2022-01-11 02:58:00
inserted FLT 2022-01-11 02:59:00
inserted FLT 2022-01-11 03:00:00
inserted FLT 2022-01-11 03:01:00
inserted FLT 2022-01-11 03:02:00
inserted FLT 2022-01-11 03:03:00
inserted FLT 2022-01-11 03:04:00
inserted FLT 2022-01-11 03:05:00
inserted FLT 2022-01-11 03:06:00
inserted FLT 2022-01-11 03:07:00
inserted FLT 2022-01-11 03:08:00
inserted FLT 2022-01-11 03:09:00
inserted FLT 2022-01-11 03:10:00
inserted FLT 2022-01-11 03:11:00
inserted FLT 2022-01-11 03:12:00
inserted FLT 2022-01-11 03:13:00
inserted FLT 2022-01-11 03:14:00
inserted FLT 2022-01-11 03:15:00
inserted FLT 2022-01-11 03:16:00
inserted FLT 2022-01-11 03:17:00
inserted FLT 2022-01-11 03:18:00
inserted FLT 2022-01-11 03:19:00
inserted FLT 2022-01-11 03:20:00
inserted FLT 2022-01-11 03:21:00
inserted FLT 2022-01-11 03:22:00
inserted FLT 2022-01-11 03:23:00
inserted FLT 2022-01-11 03:24:00
inserted F

...

inserted BHP 2022-01-20 04:08:00
inserted BHP 2022-01-20 04:09:00
inserted BHP 2022-01-20 04:10:00
inserted BHP 2022-01-20 04:11:00
inserted BHP 2022-01-20 04:12:00
inserted BHP 2022-01-20 04:13:00
inserted BHP 2022-01-20 04:14:00
inserted BHP 2022-01-20 04:15:00
inserted BHP 2022-01-20 04:16:00
inserted BHP 2022-01-20 04:17:00
inserted BHP 2022-01-20 04:18:00
inserted BHP 2022-01-20 04:19:00
inserted BHP 2022-01-20 04:20:00
inserted BHP 2022-01-20 04:21:00
inserted BHP 2022-01-20 04:22:00
inserted BHP 2022-01-20 04:23:00
inserted BHP 2022-01-20 04:24:00
inserted BHP 2022-01-20 04:25:00
inserted BHP 2022-01-20 04:26:00
inserted BHP 2022-01-20 04:27:00
inserted BHP 2022-01-20 04:28:00
inserted BHP 2022-01-20 04:29:00
inserted BHP 2022-01-20 04:30:00
inserted BHP 2022-01-20 04:31:00
inserted BHP 2022-01-20 04:32:00
inserted BHP 2022-01-20 04:33:00
inserted BHP 2022-01-20 04:34:00
inserted BHP 2022-01-20 04:35:00
inserted BHP 2022-01-20 04:36:00
inserted BHP 2022-01-20 04:37:00
inserted B

inserted BHP 2022-01-24 00:18:00
inserted BHP 2022-01-24 00:19:00
inserted BHP 2022-01-24 00:20:00
inserted BHP 2022-01-24 00:21:00
inserted BHP 2022-01-24 00:22:00
inserted BHP 2022-01-24 00:23:00
inserted BHP 2022-01-24 00:24:00
inserted BHP 2022-01-24 00:25:00
inserted BHP 2022-01-24 00:26:00
inserted BHP 2022-01-24 00:27:00
inserted BHP 2022-01-24 00:28:00
inserted BHP 2022-01-24 00:29:00
inserted BHP 2022-01-24 00:30:00
inserted BHP 2022-01-24 00:31:00
inserted BHP 2022-01-24 00:32:00
inserted BHP 2022-01-24 00:33:00
inserted BHP 2022-01-24 00:34:00
inserted BHP 2022-01-24 00:35:00
inserted BHP 2022-01-24 00:36:00
inserted BHP 2022-01-24 00:37:00
inserted BHP 2022-01-24 00:38:00
inserted BHP 2022-01-24 00:39:00
inserted BHP 2022-01-24 00:40:00
inserted BHP 2022-01-24 00:41:00
inserted BHP 2022-01-24 00:42:00
inserted BHP 2022-01-24 00:43:00
inserted BHP 2022-01-24 00:44:00
inserted BHP 2022-01-24 00:45:00
inserted BHP 2022-01-24 00:46:00
inserted BHP 2022-01-24 00:47:00
inserted B

inserted BHP 2022-01-25 01:50:00
inserted BHP 2022-01-25 01:51:00
inserted BHP 2022-01-25 01:52:00
inserted BHP 2022-01-25 01:53:00
inserted BHP 2022-01-25 01:54:00
inserted BHP 2022-01-25 01:55:00
inserted BHP 2022-01-25 01:56:00
inserted BHP 2022-01-25 01:57:00
inserted BHP 2022-01-25 01:58:00
inserted BHP 2022-01-25 01:59:00
inserted BHP 2022-01-25 02:00:00
inserted BHP 2022-01-25 02:01:00
inserted BHP 2022-01-25 02:02:00
inserted BHP 2022-01-25 02:03:00
inserted BHP 2022-01-25 02:04:00
inserted BHP 2022-01-25 02:05:00
inserted BHP 2022-01-25 02:06:00
inserted BHP 2022-01-25 02:07:00
inserted BHP 2022-01-25 02:08:00
inserted BHP 2022-01-25 02:09:00
inserted BHP 2022-01-25 02:10:00
inserted BHP 2022-01-25 02:11:00
inserted BHP 2022-01-25 02:12:00
inserted BHP 2022-01-25 02:13:00
inserted BHP 2022-01-25 02:14:00
inserted BHP 2022-01-25 02:15:00
inserted BHP 2022-01-25 02:16:00
inserted BHP 2022-01-25 02:17:00
inserted BHP 2022-01-25 02:18:00
inserted BHP 2022-01-25 02:19:00
inserted B

inserted BHP 2022-01-27 03:33:00
inserted BHP 2022-01-27 03:34:00
inserted BHP 2022-01-27 03:35:00
inserted BHP 2022-01-27 03:36:00
inserted BHP 2022-01-27 03:37:00
inserted BHP 2022-01-27 03:38:00
inserted BHP 2022-01-27 03:39:00
inserted BHP 2022-01-27 03:40:00
inserted BHP 2022-01-27 03:41:00
inserted BHP 2022-01-27 03:42:00
inserted BHP 2022-01-27 03:43:00
inserted BHP 2022-01-27 03:44:00
inserted BHP 2022-01-27 03:45:00
inserted BHP 2022-01-27 03:46:00
inserted BHP 2022-01-27 03:47:00
inserted BHP 2022-01-27 03:48:00
inserted BHP 2022-01-27 03:49:00
inserted BHP 2022-01-27 03:50:00
inserted BHP 2022-01-27 03:51:00
inserted BHP 2022-01-27 03:52:00
inserted BHP 2022-01-27 03:53:00
inserted BHP 2022-01-27 03:54:00
inserted BHP 2022-01-27 03:55:00
inserted BHP 2022-01-27 03:56:00
inserted BHP 2022-01-27 03:57:00
inserted BHP 2022-01-27 03:58:00
inserted BHP 2022-01-27 03:59:00
inserted BHP 2022-01-27 04:00:00
inserted BHP 2022-01-27 04:01:00
inserted BHP 2022-01-27 04:02:00
inserted B

inserted BHP 2022-01-28 04:57:00
inserted BHP 2022-01-28 04:58:00
inserted BHP 2022-01-28 04:59:00
inserted BHP 2022-01-28 05:00:00
inserted BHP 2022-01-28 05:11:00
👍 Gucci!


### Print out the tables which need updating

In [41]:
today = datetime.today().date()


for i in ric:
    ticker = i.replace('.AX', '')
    max_date = pd.read_sql('select max(time) from {}'.format(ticker), conn).values[0][0]
    max_date = pd.to_datetime(max_date)
    print('Last updated {} on {}'.format(ticker, max_date),'| Today is', today)
    if max_date != today:
        print(ticker, 'needs updating!')

Last updated FLT on 2022-01-28 05:11:00 | Today is 2023-01-05
FLT needs updating!
Last updated NCM on 2022-01-28 05:11:00 | Today is 2023-01-05
NCM needs updating!
Last updated BHP on 2022-01-28 05:11:00 | Today is 2023-01-05
BHP needs updating!


### **Update tables in database**

In [42]:
today = datetime.today().date()


for i in ric:
    ticker = i.replace('.AX', '')
    max_date = pd.read_sql('select max(time) from {}'.format(ticker), conn).values[0][0]
    max_date = pd.to_datetime(max_date)

    while True:
        try:
            if max_date != today:
                new_data = ek.get_timeseries(
                    i,
                    start_date = str(max_date),
                    end_date=str(max_date+timedelta(days=5)),
                    interval='minute',
                    calendar='tradingdays')

                sql = '''INSERT INTO {}(time, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s)'''.format(ticker)

                for idx,row in new_data[new_data.index > max_date].dropna().iterrows():
                    cur.execute(sql, (idx, row.OPEN, row.HIGH, row.LOW, row.CLOSE, row.VOLUME))
                    print('inserted', idx, ticker, '| Days remaining: {}'.format(str(today - new_data.index[-1].date()).date()))
                    max_date = (new_data.index[-1])

            else:
                print('Todays date: ', today)
                print('{} Last update: '.format(ticker), max_date)
                print('{} Data is up to date!'.format(ticker))
                break

        except:
            print('😢')
            break


        

😢
😢
😢


# Close connection

In [43]:
conn.close()

# Psql shell
`\l` -> View all of the defined databases on the server.

 

`\connect database_name` or `\c database_name` -> connect to db

 

`\dt`  -> Shows all tables in the current schema

---

Programmatically (or from the psql interface too, of course):

`SELECT * FROM pg_catalog.pg_tables;`
The system tables live in the pg_catalog database.




